# ECON 490: Stata Essentials (3)

## Prerequisites 

1. Understand how to effectively use Stata do files and know how to generate log files.

## Learning Outcomes

1. View the characteristics of any dataset using the command `describe`.
2. Use `help` to learn best how to run commands.
3. Understand the Stata command syntax using the command `summarize`.
4. Create loops using the commands `for`, `while`, `forvalues` and `foreach` .

## 3.1 Describing Your Data

Let's start by opening a dataset that was provided when you installed Stata onto your own computer. We will soon move on to importing our own data, but this Stata data set will help get us started. This is a dataset on automobiles and their characteristics. You can install this dataset by running the command in the cell below:

In [ ]:
sysuse auto.dta, clear

We can begin by checking the characteristics of the data set we have just downloaded. The command `describe` allows us to see the number of observations, the number of variables, a list of variable names and descriptions, and the variable types and labels of that data set.

In [ ]:
describe 

Notice that this data set consists of 12 variables and 74 observations. We can see that the first variable is named _make_, which indicates the make and model of the vehicle. We can also see that the variable _make_ is a string variables (made up of text). Other variables in this data set are numeric. For example, the variable _mpg_ indicates the vehicle's mileage (miles per gallon) as an integer. The variable _foreign_ is also numeric, and it only takes the values 0 or 1, indicating whether the car is foreign or domestically made; this is a dummy variable. 

## 3.2 Introduction to Stata Command Syntax

#### 3.2.1 Using HELP to understand commands

To help us get comfortable with the syntax used by Stata, let's start with a simple and useful command: `summarize`. This command will give us the basic statistics from any variable(s) in the data set, such as the variables we have discussed above. 

In [ ]:
summarize make mpg foreign

Stata has a help manual installed in the program which provides documentation for all Stata published commands. This information can be reached by typing the command `help` and then the name of the command we need extra information about. 

To see the extra information that is available by using `summarize`, run the command below: 

In [ ]:
help summarize

You will need to run this command directly into the Stata console on your computer in order to able to see all of the information provided by `help`. You should run this command now in order to be able to see that output directly.

When you do you will see that the first 1-2 letters of the command are often underlined. This underlining indicates the shortest permitted abbreviation for a command (or option). 

For example, if you type `help rename` you will see that `rename` can be abbreviated `ren`, `rena`, or `renam`, or it can be spelled out in its entirety. 

Other examples are, `g`enerate, `ap`pend, `rot`ate, `ru`n.

If there is no underline, then no abbreviation is allowed. For example, the command `replace` cannot be abbreviated. The reason for this is that Stata doesn't want us to accidentally make changes to our data by replacing the information in the variable. 

We can write the `summarize` command with its shortest abbreviation `su` or a longer abbreviation such as `sum`. 

Also, in the Stata help output we can see that some words are written in blue and are encased within square brackets. We will talk more about these options below, but in Stata we can directly click on those links for more information from help. 

Finally, help provides a list of the available options for a command. In the case of `summarize`, these options allow us to display extra information for a variable. We will learn more about this below in section 3.2.4.

#### 3.2.2 Imposing IF conditions

When the syntax of the command allows for `[if]`, we can run the command on a subset of the data that satisfies any condition we choose. Here is the list of conditional operators available to us:

1. Equal: ==
2. Greater than and less than: > and <
3. Greater than or equal and less than or equal: >= and <= 
4. Not Equal: != 

We can also compound different conditions using the list of logical operators:

1. And: & 
2. Or: | 
3. Not: ! or ~ 

Let's look at an example which applies this new knowledge: summarizing the variable _price_ when the make of the car is domestic (i.e. not foreign):

In [ ]:
su price if foreign == 0

Let's do this again, but now we will impose the additional condition that the mileage must be less than 25.

In [ ]:
su price if foreign == 0  & mpg < 25

Maybe we want to restrict to a particular list of values. Here we can make use of the option `inlist()` or we can write out all of the conditions using the "or" operator:

In [ ]:
su price if inlist(mpg,10,15,25,40)

This works exactly the same way as this command:

In [ ]:
su price if mpg == 10 | mpg == 15 | mpg == 25 | mpg == 40

Maybe we want to restrict to values in a particular range. Here we can make use of the option `inrange()` or we can write out all of the conditions using the conditional operators:

In [ ]:
su price if inrange(mpg,5,25) 

Notice the output returned by the code below is equal to the previous cell:

In [ ]:
su price if mpg >= 5 & mpg <= 25

There might be variables for which there is no information recorded for some observations. For example, when we `summarize` our automobile data we will see that there are 74 observations for most variables, but that the variable _rep78_ has only 69 observations - for five observations there is no repair record indicated in the data set.

In [ ]:
su price rep78 

If, for some reason, we only want to consider observations without missing values, we can use the option `!missing()` which combines the command `missing()` with the negative conditional operator "!". For example, the command below says to summarize the variable _price_ for all observations for which _rep78_ is NOT missing.

In [ ]:
su price if !missing(rep78)

This command can also be written using the conditional operator since missing numeric variables are indicated by a ".". This is shown below:

In [ ]:
su price if rep78 != .

Notice that in both cases there are only 69 observations.

If we wanted to do this with missing string variables, we could indicate those with "". 

#### 3.2.3 Imposing IN conditions 

We can also subset the data by using the observation number. The example below summarizes the data in observations 1 through 10.

In [ ]:
su price in 1/10

But be careful! This type of condition is generally not recommended because it depends on the order of the data.

To see this, lets sort the observations in ascending order by running the command `sort`: 

In [ ]:
sort price 
su price in 1/10

We can see that the result changes because the observations 1 through 10 in the data are now different.

Always avoid using `in` whenever you can. Try to use `if` instead!

#### 3.2.4 Command options

When we used the `help` command, we saw that we can introduce some optional arguments after a comma. In the case of the `summarize` command we were shown the following options: `d`etail, `mean`only, `f`ormat and `sep`arator(#). 

If we want additional statistics apart from the mean, standard deviation, min, and max values, we can use the option `detail` or just `d` for short.

In [ ]:
su price , d

## 3.3 Using Loops 

Much like any other programming language, there are `for` and `while` loops that we can use to iterate through many times. In particular, the `for` loops are also sub-divided into `forvalues` (which iterate across a range of numbers) and `foreach` (which iterate across a list of names). 

It is very common that these loops create a local scope (i.e. the iteration labels only exist within a loop). A local in Stata is a special variable that temporarily stores information. We'll discuss locals in the next module, but consider this simple example in which the letter "i" is used as a place holder for the number 95. 

In [ ]:
local i = 95

display `i'

We can also create locals that are strings rather than numeric in type. Consider this example:

In [ ]:
local course = "ECON 490"

display "`course'"


We can store anything inside a local. When we want to use that information, we include the local encased in a backtick (\`) and  apostrophe (').

In [ ]:
local course = "ECON 490"

display "I am enrolled in `course' and hope my grade will be `i'%!"

#### 3.3.1 Creating loops Using `forvalues` 

Whenever we want to iterate across a range of values defined as `forvalues` = `local_var_name' =  min_value(steps)max_value, we can write the command below. Here we are iterating from 1 to 10 in increments of 1.

In [ ]:
forvalues counter=1(1)10{
    *Notice that now counter is a local variable
    display `counter'
}

Notice that the open brace `{` needs to be on the same line as the `for` command, with no comments after it. Similarly, the closing brace `}` needs to be on its own line.

Experiment below with the command above by changing the increments and min or max values. See what your code outputs.

#### 3.3.2 Creating loops using `foreach`

Whenever we want to iterate across a list of names, we can write the command below which asks Stata to `summarize` for a list of variables (in this example, _mpg_ and _price_).

In [ ]:
foreach name in "mpg" "price"{
    summarize `name'
}

We can have a list stored in a local variable as well.

In [ ]:
local namelist "mpg price"
foreach name in `namelist'{
    summarize `name'
}

#### 3.3.3 Writing loops with conitions using `while`

Whenever we want to iterate until a condition is met, we can write the command below. The condition here is simply "while counter is less than 5". 

In [ ]:
local counter = 1 
while `counter'<5{
    display `counter'
    local counter = `counter'+1
}

## 3.4 Wrap Up

In this module, we looked at the way Stata commands function and how their syntax works. In general, many Stata commands will follow the folllowing structure: 

name_of_command [varlist] [if] [in] [weight] [, options]

At this point, you should feel more comfortable reading a documentation file for a Stata command. The question that remains is how to find new commands!

You are encouraged to search for commands using the command `search`. For example, if you are interested in running a regression you can write:

In [ ]:
search regress 

You will see that a new Stata window pops up on your computer, and you can click on the different options that it shows to look at the documentation for all these commands. Try it yourself in the code cell below!

In the following modules, whenever there is a command which confuses you, feel free to write `search command` or `help command` to redirect to the documentation for reference.

> Remember: These commands have to be used on your Stata console!

In the next module, we will expand on our knowledge of locals, as well as globals, another type of variable.

## References

[PDF documentation in Stata](https://www.youtube.com/watch?v=zyJ8Wk3rV2c&list=PLN5IskQdgXWnnIVeA_Y0OBGmnw21fvcmU&index=2) <br>
[Stata Interface tour](https://www.youtube.com/watch?v=FQ1MBQw_MTI) <br>
[One-way tables of summary statistics](https://www.youtube.com/watch?v=ug0LihyIzvM) <br>
[Two-way tables of summary statistics](https://www.youtube.com/watch?v=u_Efw1oWxWk)